In [3]:
import tensorflow as tf
from datasets import load_dataset
from transformers import TFMT5ForConditionalGeneration, MT5Tokenizer, DataCollatorForSeq2Seq
from tensorflow.keras.optimizers import Adam


In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
tokenizer = MT5Tokenizer.from_pretrained("google/mt5-small")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [5]:
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd

# Hugging Face'ten veri setini yükleme
dataset = load_dataset("batubayk/TR-News")  # Kendi veri setinizin adını yazın

# Train ve validation setlerini alın
train_dataset = dataset['train']
val_dataset = dataset['validation']
test_dataset = dataset['test']
# Train dataset'in yarısını alma
train_half = train_dataset.select(range(len(train_dataset) // 3))


# Tüm veri setlerini bir DatasetDict'e dönüştürme
combined_datasets = DatasetDict({
    'train': train_half,       # Yarıya indirgenmiş train set
    'validation': val_dataset, # Validation set
    'test': test_dataset       # Test set
})

# Birleştirilmiş veri setini görüntüleme
print(combined_datasets)


README.md:   0%|          | 0.00/614 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/686M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/36.4M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/37.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/277573 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/14610 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/15379 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['abstract', 'author', 'content', 'date', 'source', 'tags', 'title', 'topic', 'url'],
        num_rows: 92524
    })
    validation: Dataset({
        features: ['abstract', 'author', 'content', 'date', 'source', 'tags', 'title', 'topic', 'url'],
        num_rows: 14610
    })
    test: Dataset({
        features: ['abstract', 'author', 'content', 'date', 'source', 'tags', 'title', 'topic', 'url'],
        num_rows: 15379
    })
})


In [6]:
def tokenize_sample_data(data):

    input_feature = tokenizer(data['content'], truncation=True, max_length=1024)
    label = tokenizer(data['abstract'], truncation=True, max_length=128)
    return {
        "input_ids" : input_feature['input_ids'],
        "attention_mask" : input_feature['attention_mask'],
        "labels" : label['input_ids'],
    }

tokenized_ds = combined_datasets.map(
    tokenize_sample_data,
    remove_columns= ['abstract','author','content','date','source','tags','title','topic','url'],
    batched=True,
    batch_size= 512
)
tokenized_ds

Map:   0%|          | 0/92524 [00:00<?, ? examples/s]

Map:   0%|          | 0/14610 [00:00<?, ? examples/s]

Map:   0%|          | 0/15379 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 92524
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 14610
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 15379
    })
})

In [ ]:
'''

from transformers import MT5Tokenizer, MT5ForConditionalGeneration

# 1. Tokenizer ve modelin yüklenmesi
model_name = "google/mt5-small"
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)

# 2. Özetlenmek istenen Türkçe metin
turkish_text = """
İklim değişikliği, küresel sıcaklıkların artışıyla birlikte ortaya çıkan çevresel, sosyal ve ekonomik sorunlara neden olmaktadır.
Özellikle kuraklık, sel ve orman yangınları gibi doğal afetlerin sıklığı artarken, tarımsal üretimde de ciddi düşüşler yaşanmaktadır.
Bu durum, gıda güvenliğini tehdit etmekte ve toplumları olumsuz yönde etkilemektedir.
"""

# 3. Özetleme için giriş formatı: 'summarize:' ön ekini ekliyoruz
input_text = f"summarize: {turkish_text}"

# 4. Tokenizer ile metni encode etme
input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

# 5. Modeli kullanarak özetleme yapma
summary_ids = model.generate(
    input_ids,
    max_length=150,
    min_length=30,
    length_penalty=2.0,
    num_beams=4,
    early_stopping=True
)

# 6. Özetlenen metni decode etme
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# 7. Sonucu yazdırma
print("Özetlenen Metin:")
print(summary)
'''

'\n\nfrom transformers import MT5Tokenizer, MT5ForConditionalGeneration\n\n# 1. Tokenizer ve modelin yüklenmesi\nmodel_name = "google/mt5-small"\ntokenizer = MT5Tokenizer.from_pretrained(model_name)\nmodel = MT5ForConditionalGeneration.from_pretrained(model_name)\n\n# 2. Özetlenmek istenen Türkçe metin\nturkish_text = """\nİklim değişikliği, küresel sıcaklıkların artışıyla birlikte ortaya çıkan çevresel, sosyal ve ekonomik sorunlara neden olmaktadır.\nÖzellikle kuraklık, sel ve orman yangınları gibi doğal afetlerin sıklığı artarken, tarımsal üretimde de ciddi düşüşler yaşanmaktadır.\nBu durum, gıda güvenliğini tehdit etmekte ve toplumları olumsuz yönde etkilemektedir.\n"""\n\n# 3. Özetleme için giriş formatı: \'summarize:\' ön ekini ekliyoruz\ninput_text = f"summarize: {turkish_text}"\n\n# 4. Tokenizer ile metni encode etme\ninput_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)\n\n# 5. Modeli kullanarak özetleme yapma\nsummary_ids = model.genera

In [7]:
import torch
from transformers import AutoConfig, AutoModelForSeq2SeqLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model konfigürasyonu
mt5_config = AutoConfig.from_pretrained(
    "google/mt5-small",
    max_length=256,
    length_penalty=1.0,
    no_repeat_ngram_size=3,
    num_beams=5,
)

# PyTorch tabanlı modeli yükleme ve GPU/CPU'ya taşıma
model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small", config=mt5_config).to(device)


pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [8]:
data_collator = DataCollatorForSeq2Seq(
  tokenizer,
  model=model,
  return_tensors="pt")

In [11]:
import evaluate
import numpy as np
from nltk.tokenize import RegexpTokenizer

rouge_metric = evaluate.load("rouge")

# define function for custom tokenization
def tokenize_sentence(arg):
    return arg  # Bu fonksiyon kaldırılabilir veya değiştirilmeden kullanılabilir

# define function to get ROUGE scores with custom tokenization
def metrics_func(eval_arg):
  preds, labels = eval_arg
  # Replace -100
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  # Convert id tokens to text
  text_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
  text_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
  # Insert a line break (\n) in each sentence for ROUGE scoring
  # (Note : Please change this code, when you perform on other languages except for Japanese)
  text_preds = [(p if p.endswith((".", "!", "?")) else p + ".") for p in text_preds]
  text_labels = [(l if l.endswith((".", "!", "?")) else l + ".") for l in text_labels]

  # Türkçe cümle tokenizasyonu
  sent_tokenizer_tr = RegexpTokenizer(r'[^.!?]*[.!?]')
  text_preds = ["\n".join(np.char.strip(sent_tokenizer_tr.tokenize(p))) for p in text_preds]
  text_labels = ["\n".join(np.char.strip(sent_tokenizer_tr.tokenize(l))) for l in text_labels]

  # compute ROUGE score with custom tokenization
  return rouge_metric.compute(
    predictions=text_preds,
    references=text_labels,
    tokenizer=tokenize_sentence
  )

In [9]:
!pip install evaluate
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=fbd005ae5ff425f81cb4eafcea1a8325c1777190e03e83bfe1020132285ca48f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [12]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="mt5-summarize-ja",
    log_level="error",
    num_train_epochs=5,
    learning_rate=2e-5,
    lr_scheduler_type="linear",
    warmup_steps=250,
    optim="adafactor",
    weight_decay=0.01,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    evaluation_strategy="steps",
    eval_steps=200,
    predict_with_generate=True,
    generation_max_length=256,
    save_steps=500,
    logging_steps=50,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [14]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    compute_metrics=metrics_func,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"].select(range(1000)),
)

trainer.train()


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# Eğitilen modeli kaydet
output_dir = "./trained_model"  # Kaydedilecek dizin
trainer.save_model(output_dir)  # Modeli ve tokenizer'ı kaydeder

# Tokenizer'ı ayrıca kaydetmek isterseniz
tokenizer.save_pretrained(output_dir)

print(f"Model ve tokenizer {output_dir} dizinine kaydedildi.")

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Eğitilen modeli ve tokenizer'ı yükle
model = AutoModelForSeq2SeqLM.from_pretrained("./trained_model")
tokenizer = AutoTokenizer.from_pretrained("./trained_model")


In [ ]:
test_text = "Dünyamızda 1900’lü yıllardan günümüze kadar dil öğretiminde çeşitli metinler kullanılmıştır. Bunlar “ edebi metinler, üretilmiş metinler, özgün ve özel metinler ” başlıkları altında toplanmıştır. Metinlerin seçimi dil öğretim yaklaşım ve yöntemlerine göre değişmektedir. Her yaklaşım kendine özgü metin kullanmıştır. Geleneksel yaklaşımda dil bilgisi kuralları, atasözleri, edebiyat, genel kültür gibi konuların öğretimine ağırlık verildiğinden edebi metinler kullanılmıştır. Davranışçı yaklaşımda dil davranış olarak ele alınmış, tekrar, taklit ve ezberleme yoluyla öğretilmiştir. Bu yaklaşımda edebi metinler yerine üretilmiş metinler kullanılmıştır. Bilişsel yaklaşımda “dil iletişim aracıdır” görüşü yayılmış ve özgün metinler kullanılmaya başlanmıştır. "


In [ ]:
'''inputs = tokenizer(
    test_text,
    return_tensors="pt",  # PyTorch tensörleri olarak döner
    max_length=1024,      # Modelin giriş sınırı
    truncation=True       # Çok uzun metinleri keser
)
'''

In [ ]:
# Özetleme
output_ids = model.generate(
    inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_length=128,  # Özet uzunluğu sınırı
    num_beams=4,     # Beam search kullanımı
    no_repeat_ngram_size=2,  # Tekrar eden n-gramları engeller
    length_penalty=0.6       # Kısa özetlere öncelik
)

# Özet çözümleme
summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("Özet:", summary)


In [ ]:
'''from google.colab import drive
drive.mount('/content/drive')'''